# Ejemplo

d) Select five assets from the market, calculate their returns, and, with this, calculate the expected return and
the covariance matrix. Once these parameters are obtained, the optimization problem will be solved, and
investment recommendations will be generated. Remember to verify that each of the assets has sufficient
liquidity.

## Fórmula para $w$ que optimiza el ratio de Sharpe
$$
w = \frac{\Sigma^{-1} (\mu - r_f 1)}{1^T \Sigma^{-1} (\mu - r_f 1)}
$$

- $w = [w_1,w_2,...,w_n]$ : vector de pesos de los activos del portafolio
- $\mu = [\mu_1,\mu_2,...,\mu_3]$ : vector del valor esperado de cada uno de los activos
- $\Sigma$: la matriz de covarianza de los rendimientos de los activos
- $r_f$: la tasa libre de riesgo
- $1$: vector de unos

## Código

### Librerías

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from IPython.display import display, Markdown
from ipywidgets import interact, FloatSlider

### Obtención de datos

In [3]:
tickers=['AMZN','WMT','GOOGL','AAPL','JPM']

f_inicial='2020-01-01'
f_final='2024-11-10'

datos1=yf.download(tickers,f_inicial,f_final)['Adj Close']

rt = datos1.pct_change().dropna()

[*********************100%***********************]  5 of 5 completed
